# 06 - Production Inference with Pydantic AI

This notebook demonstrates production-ready inference using the fine-tuned model with Pydantic AI for validation and grammar constraints for guaranteed valid JSON.

## What we'll do:
1. Load the fine-tuned model
2. Set up Pydantic schemas with validation
3. Integrate with outlines for grammar-constrained generation
4. Create production-ready inference pipeline
5. Add confidence scoring
6. Build helper utilities

## 1. Setup and Imports

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel
from pydantic import BaseModel, Field, field_validator, ValidationError
import json
import re
from typing import Optional
import warnings
warnings.filterwarnings('ignore')

print("✓ Imports successful")

## 2. Define Pydantic Schema with Validation

In [ ]:
class ProjectEstimation(BaseModel):
    """Schema for project estimation extraction from BRDs."""
    
    effort_hours: float = Field(
        gt=0,
        description="Total project effort in hours"
    )
    timeline_weeks: int = Field(
        gt=0,
        le=520,
        description="Project timeline in weeks"
    )
    cost_usd: float = Field(
        gt=0,
        description="Estimated project cost in USD"
    )
    
    @field_validator('timeline_weeks')
    @classmethod
    def validate_timeline(cls, v):
        if v > 104:  # 2 years
            raise ValueError('Timeline exceeds reasonable range for typical projects')
        return v
    
    @field_validator('cost_usd')
    @classmethod
    def validate_cost(cls, v, info):
        effort = info.data.get('effort_hours')
        if effort and v / effort < 10:
            raise ValueError('Cost per hour too low (minimum $10/hour)')
        return v
    
    def to_dict(self):
        """Convert to dictionary."""
        return self.model_dump()
    
    def to_json(self):
        """Convert to JSON string."""
        return self.model_dump_json(indent=2)

# Test the schema
test_estimation = ProjectEstimation(
    effort_hours=480.0,
    timeline_weeks=12,
    cost_usd=48000.0
)

print("Example validated schema:")
print(test_estimation.to_json())
print("\n✓ Pydantic schema defined")

## 3. Load Fine-tuned Model

In [ ]:
BASE_MODEL_ID = "meta-llama/Llama-3.2-1B"
FINETUNED_MODEL_DIR = "../models/final/llama-3.2-1b-brd-final"

print("Loading fine-tuned model...\n")

# Quantization config for inference
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

# Load LoRA weights
model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL_DIR)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("✓ Fine-tuned model loaded")
print(f"  Memory footprint: {model.get_memory_footprint() / 1e9:.2f} GB")

## 4. Create Inference Pipeline

In [ ]:
class BRDExtractor:
    """
    Production-ready BRD extraction pipeline with Pydantic validation.
    """
    
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.instruction = """Extract the project estimation fields from the following Business Requirements Document.
Return a JSON object with these exact fields: effort_hours (number), timeline_weeks (number), cost_usd (number).
Return ONLY the JSON object, no additional text."""
    
    def _create_prompt(self, brd_text: str) -> str:
        """Create formatted prompt from BRD text."""
        return f"""### Instruction:
{self.instruction}

### Input:
{brd_text}

### Output:
"""
    
    def _generate(self, prompt: str, max_tokens: int = 150, temperature: float = 0.1) -> str:
        """Generate text using the model."""
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=temperature,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
            )
        
        generated = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return generated
    
    def _extract_json(self, text: str) -> Optional[dict]:
        """Extract JSON from generated text."""
        try:
            # Extract output section
            if "### Output:" in text:
                text = text.split("### Output:")[-1].strip()
            
            # Find JSON object
            match = re.search(r'\{[^}]+\}', text)
            if match:
                json_str = match.group(0)
                return json.loads(json_str)
            return None
        except Exception:
            return None
    
    def extract(self, brd_text: str, validate: bool = True) -> dict:
        """
        Extract project estimation from BRD text.
        
        Args:
            brd_text: The BRD document text
            validate: Whether to validate with Pydantic schema
        
        Returns:
            Dictionary with extraction results and metadata
        """
        # Generate prompt
        prompt = self._create_prompt(brd_text)
        
        # Generate output
        generated = self._generate(prompt)
        
        # Extract JSON
        extracted_json = self._extract_json(generated)
        
        result = {
            "success": False,
            "data": None,
            "validated": False,
            "errors": [],
            "raw_output": generated,
        }
        
        if extracted_json is None:
            result["errors"].append("Failed to extract valid JSON from output")
            return result
        
        result["data"] = extracted_json
        result["success"] = True
        
        # Validate with Pydantic if requested
        if validate:
            try:
                validated = ProjectEstimation(**extracted_json)
                result["validated"] = True
                result["data"] = validated.to_dict()
            except ValidationError as e:
                result["validated"] = False
                result["errors"].append(f"Pydantic validation failed: {str(e)}")
        
        return result

# Initialize extractor
extractor = BRDExtractor(model, tokenizer)

print("✓ BRD Extractor initialized")

## 5. Test the Extraction Pipeline

In [ ]:
# Sample BRD for testing
sample_brd = """Business Requirements Document
Project: E-commerce Mobile Application

Overview:
We require a cross-platform mobile application (iOS and Android) for our e-commerce business. 
The app will feature product browsing, shopping cart functionality, secure checkout, 
order tracking, and user account management.

Technical Scope:
- React Native development
- Integration with existing REST API
- Payment gateway integration (Stripe)
- Push notifications
- Analytics integration

Resource Requirements:
- 2 senior mobile developers
- 1 UI/UX designer
- 1 QA engineer

Timeline: 
The project is estimated to take 16 weeks from kickoff to production deployment.

Effort Estimation:
Total development effort is estimated at 960 hours across all team members.

Budget:
The total project cost is projected at $120,000, including all development, 
design, testing, and deployment activities.
"""

print("Extracting from sample BRD...\n")
print("="*80)
print("INPUT BRD:")
print("-"*80)
print(sample_brd)
print("="*80)

# Extract
result = extractor.extract(sample_brd, validate=True)

print("\nEXTRACTION RESULT:")
print("="*80)
print(f"Success: {result['success']}")
print(f"Validated: {result['validated']}")
print(f"\nExtracted Data:")
print(json.dumps(result['data'], indent=2))

if result['errors']:
    print(f"\nErrors: {result['errors']}")

print("="*80)

## 6. Batch Processing

In [ ]:
def batch_extract(brd_texts: list, extractor: BRDExtractor) -> list:
    """
    Process multiple BRDs in batch.
    """
    from tqdm.notebook import tqdm
    
    results = []
    for brd in tqdm(brd_texts, desc="Processing BRDs"):
        result = extractor.extract(brd, validate=True)
        results.append(result)
    
    return results

# Example: Process multiple BRDs
sample_brds = [
    """BRD for CRM System: We need a customer relationship management system. 
    3 developers, 8 weeks, 480 hours total, $60,000 budget.""",
    
    """Website Redesign Project: Complete redesign of corporate website with new CMS. 
    Timeline: 10 weeks. Effort: 400 hours. Cost: $50,000.""",
    
    """Data Migration Project: Migrate legacy database to cloud. 
    2 engineers, 6 weeks, estimated 240 hours, budget of $30,000.""",
]

print("Processing batch of BRDs...\n")
batch_results = batch_extract(sample_brds, extractor)

# Display results
print("\nBatch Processing Results:")
print("="*80)
for i, result in enumerate(batch_results, 1):
    print(f"\nBRD {i}:")
    if result['success'] and result['validated']:
        print(f"  ✓ Success")
        print(f"  Data: {result['data']}")
    else:
        print(f"  ✗ Failed")
        print(f"  Errors: {result['errors']}")

print("="*80)

## 7. Save Extraction Pipeline for Reuse

In [ ]:
# Save the extractor class to a Python file for easy import
extractor_code = '''"""BRD Extraction Pipeline with Pydantic Validation

Usage:
    from brd_extractor import BRDExtractor, ProjectEstimation
    from transformers import AutoModelForCausalLM, AutoTokenizer
    from peft import PeftModel
    
    # Load model
    base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B", ...)
    model = PeftModel.from_pretrained(base_model, "path/to/finetuned")
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
    
    # Extract
    extractor = BRDExtractor(model, tokenizer)
    result = extractor.extract(brd_text)
"""

import torch
from pydantic import BaseModel, Field, field_validator, ValidationError
import json
import re
from typing import Optional

class ProjectEstimation(BaseModel):
    """Schema for project estimation extraction from BRDs."""
    
    effort_hours: float = Field(gt=0, description="Total project effort in hours")
    timeline_weeks: int = Field(gt=0, le=520, description="Project timeline in weeks")
    cost_usd: float = Field(gt=0, description="Estimated project cost in USD")
    
    @field_validator('timeline_weeks')
    @classmethod
    def validate_timeline(cls, v):
        if v > 104:
            raise ValueError('Timeline exceeds reasonable range for typical projects')
        return v
    
    @field_validator('cost_usd')
    @classmethod
    def validate_cost(cls, v, info):
        effort = info.data.get('effort_hours')
        if effort and v / effort < 10:
            raise ValueError('Cost per hour too low (minimum $10/hour)')
        return v
    
    def to_dict(self):
        return self.model_dump()
    
    def to_json(self):
        return self.model_dump_json(indent=2)

class BRDExtractor:
    """Production-ready BRD extraction pipeline."""
    
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.instruction = """Extract the project estimation fields from the following Business Requirements Document.
Return a JSON object with these exact fields: effort_hours (number), timeline_weeks (number), cost_usd (number).
Return ONLY the JSON object, no additional text."""
    
    def _create_prompt(self, brd_text: str) -> str:
        return f"""### Instruction:\n{self.instruction}\n\n### Input:\n{brd_text}\n\n### Output:\n"""
    
    def _generate(self, prompt: str, max_tokens: int = 150, temperature: float = 0.1) -> str:
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=temperature,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
            )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    def _extract_json(self, text: str) -> Optional[dict]:
        try:
            if "### Output:" in text:
                text = text.split("### Output:")[-1].strip()
            match = re.search(r'\\{[^}]+\\}', text)
            if match:
                return json.loads(match.group(0))
            return None
        except Exception:
            return None
    
    def extract(self, brd_text: str, validate: bool = True) -> dict:
        prompt = self._create_prompt(brd_text)
        generated = self._generate(prompt)
        extracted_json = self._extract_json(generated)
        
        result = {
            "success": False,
            "data": None,
            "validated": False,
            "errors": [],
            "raw_output": generated,
        }
        
        if extracted_json is None:
            result["errors"].append("Failed to extract valid JSON from output")
            return result
        
        result["data"] = extracted_json
        result["success"] = True
        
        if validate:
            try:
                validated = ProjectEstimation(**extracted_json)
                result["validated"] = True
                result["data"] = validated.to_dict()
            except ValidationError as e:
                result["validated"] = False
                result["errors"].append(f"Pydantic validation failed: {str(e)}")
        
        return result
'''

# Save to file
with open("../data/processed/brd_extractor.py", "w") as f:
    f.write(extractor_code)

print("✓ Extractor saved to: ../data/processed/brd_extractor.py")
print("\nYou can now import and use it:")
print("  from brd_extractor import BRDExtractor, ProjectEstimation")

## Summary

### What we've built:
- ✓ Production-ready extraction pipeline
- ✓ Pydantic validation for type safety
- ✓ Error handling and reporting
- ✓ Batch processing capabilities
- ✓ Reusable Python module

### Key Features:
- **Type-safe**: Pydantic ensures valid data types
- **Validated**: Custom validators for business rules
- **Robust**: Handles malformed outputs gracefully
- **Production-ready**: Easy to integrate into applications
- **Modular**: Reusable across projects

### Integration Options:
1. **REST API**: Wrap in FastAPI/Flask
2. **CLI Tool**: Add argparse interface
3. **Web App**: Use with Streamlit/Gradio
4. **Batch Processing**: Process files from S3/local storage
5. **Microservice**: Deploy as Docker container

### Next Steps:
Move on to `07_demo.ipynb` for an interactive demo interface!

### Notes:
- Pydantic validation catches invalid outputs
- Fine-tuned model + validation = production-ready
- Can add more complex validation rules as needed
- Consider adding grammar constraints (outlines) for 100% valid JSON